<a href="https://colab.research.google.com/github/Anjalee01/Advent-Of-Code-2024/blob/main/Day12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def flood_fill(grid, x, y, visited, plant_type):
    """Performs flood-fill to find a region and calculates its area and perimeter."""
    rows, cols = len(grid), len(grid[0])
    stack = [(x, y)]
    area = 0
    perimeter = 0
    visited.add((x, y))

    # Directions for movement: up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    while stack:
        cx, cy = stack.pop()
        area += 1

        for dx, dy in directions:
            nx, ny = cx + dx, cy + dy
            if 0 <= nx < rows and 0 <= ny < cols:
                if grid[nx][ny] == plant_type:
                    if (nx, ny) not in visited:
                        visited.add((nx, ny))
                        stack.append((nx, ny))
                else:
                    # Edge of the region
                    perimeter += 1
            else:
                # Edge of the grid
                perimeter += 1

    return area, perimeter


def calculate_fencing_cost(grid):
    """Calculates the total fencing cost for the garden."""
    rows, cols = len(grid), len(grid[0])
    visited = set()
    total_cost = 0

    for x in range(rows):
        for y in range(cols):
            if (x, y) not in visited:
                # Start a new region for a specific plant type
                plant_type = grid[x][y]
                area, perimeter = flood_fill(grid, x, y, visited, plant_type)
                cost = area * perimeter
                total_cost += cost
                print(f"Region {plant_type}: Area={area}, Perimeter={perimeter}, Cost={cost}")

    return total_cost


def read_map(file_path):
    """Reads the garden map from a file."""
    with open(file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]


def main():
    # Read the garden map from the input file
    file_path = "input.txt"
    garden_map = read_map(file_path)

    # Calculate the total fencing cost
    total_cost = calculate_fencing_cost(garden_map)

    # Output the result
    print("Total fencing cost:", total_cost)


if __name__ == "__main__":
    main()

Region O: Area=10, Perimeter=22, Cost=220
Region K: Area=122, Perimeter=74, Cost=9028
Region T: Area=96, Perimeter=78, Cost=7488
Region C: Area=5, Perimeter=12, Cost=60
Region Y: Area=88, Perimeter=64, Cost=5632
Region G: Area=212, Perimeter=198, Cost=41976
Region M: Area=16, Perimeter=28, Cost=448
Region F: Area=1, Perimeter=4, Cost=4
Region P: Area=136, Perimeter=100, Cost=13600
Region D: Area=1, Perimeter=4, Cost=4
Region S: Area=150, Perimeter=74, Cost=11100
Region Y: Area=86, Perimeter=54, Cost=4644
Region T: Area=22, Perimeter=32, Cost=704
Region Z: Area=55, Perimeter=48, Cost=2640
Region C: Area=29, Perimeter=30, Cost=870
Region P: Area=146, Perimeter=82, Cost=11972
Region Z: Area=1, Perimeter=4, Cost=4
Region R: Area=7, Perimeter=14, Cost=98
Region Q: Area=57, Perimeter=50, Cost=2850
Region J: Area=30, Perimeter=34, Cost=1020
Region L: Area=145, Perimeter=102, Cost=14790
Region I: Area=362, Perimeter=196, Cost=70952
Region V: Area=3, Perimeter=8, Cost=24
Region J: Area=27, Peri

In [ ]:
def find_regions(grid):
    rows, cols = len(grid), len(grid[0])
    visited = set()
    regions = []

    def dfs(r, c, plant_type):
        if (r, c) in visited or r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] != plant_type:
            return set()

        cells = {(r, c)}
        visited.add((r, c))

        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            cells.update(dfs(r + dr, c + dc, plant_type))

        return cells

    for r in range(rows):
        for c in range(cols):
            if (r, c) not in visited:
                region = dfs(r, c, grid[r][c])
                if region:
                    regions.append(region)

    return regions

def count_sides(region, grid):
    rows, cols = len(grid), len(grid[0])
    sides = 0
    visited_edges = set()

    def trace_line(r, c, dr, dc):
        nr, nc = r + dr, c + dc
        return (nr < 0 or nc < 0 or nr >= rows or nc >= cols or (nr, nc) not in region)

    # Count horizontal lines
    for r, c in region:
        # Check up
        if (r, c, 'u') not in visited_edges and trace_line(r, c, -1, 0):
            start_c = c
            while (r, start_c-1) in region and trace_line(r, start_c-1, -1, 0):
                start_c -= 1
            end_c = c
            while (r, end_c+1) in region and trace_line(r, end_c+1, -1, 0):
                end_c += 1
            sides += 1
            for i in range(start_c, end_c+1):
                visited_edges.add((r, i, 'u'))

        # Check down
        if (r, c, 'd') not in visited_edges and trace_line(r, c, 1, 0):
            start_c = c
            while (r, start_c-1) in region and trace_line(r, start_c-1, 1, 0):
                start_c -= 1
            end_c = c
            while (r, end_c+1) in region and trace_line(r, end_c+1, 1, 0):
                end_c += 1
            sides += 1
            for i in range(start_c, end_c+1):
                visited_edges.add((r, i, 'd'))

    # Count vertical lines
    for r, c in region:
        # Check left
        if (r, c, 'l') not in visited_edges and trace_line(r, c, 0, -1):
            start_r = r
            while (start_r-1, c) in region and trace_line(start_r-1, c, 0, -1):
                start_r -= 1
            end_r = r
            while (end_r+1, c) in region and trace_line(end_r+1, c, 0, -1):
                end_r += 1
            sides += 1
            for i in range(start_r, end_r+1):
                visited_edges.add((i, c, 'l'))

        # Check right
        if (r, c, 'r') not in visited_edges and trace_line(r, c, 0, 1):
            start_r = r
            while (start_r-1, c) in region and trace_line(start_r-1, c, 0, 1):
                start_r -= 1
            end_r = r
            while (end_r+1, c) in region and trace_line(end_r+1, c, 0, 1):
                end_r += 1
            sides += 1
            for i in range(start_r, end_r+1):
                visited_edges.add((i, c, 'r'))

    return sides

def solve_part2(input_text):
    grid = [list(line.strip()) for line in input_text.strip().split('\n')]
    regions = find_regions(grid)

    total_price = 0
    for region in regions:
        area = len(region)
        num_sides = count_sides(region, grid)
        price = area * num_sides
        total_price += price

    return total_price

with open('input.txt', 'r') as f:
    input_text = f.read()

result = solve_part2(input_text)
print(result)

937032
